<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/ANMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplos de ANMI

In [117]:
from sympy import *
from sympy.matrices import Matrix as mat
from sympy.matrices import randMatrix
from sympy import symbols

import numpy as np
from numpy.linalg import cond as numero_condicion

from scipy.linalg import orth

## Tema 2

### Número de condicion

El número de condición se define como $\vert\vert A \vert\vert \cdot \vert\vert A^{-1}\vert\vert$. Un número de condición cercano a 1 implica una matriz más estable frente a métodos con elementos diferenciales. Se cumple que las matrices ortogonales tienen número de condición 1.

In [10]:
M = mat(((1, 2, 3), (2, 3, 1), (3, 2, 4)))
M

Matrix([
[1, 2, 3],
[2, 3, 1],
[3, 2, 4]])

In [27]:
numero_condicion(np.array(M).astype(int))

6.960250455451664

In [28]:
M_ort = (orth(np.array(M).astype(int)))
M_ort

array([[-0.50127996, -0.23313726,  0.83328592],
       [-0.45615124,  0.88953592, -0.02553206],
       [-0.73528528, -0.39290311, -0.55225239]])

In [29]:
numero_condicion(M_ort)

1.0000000000000004

### Factorización LU y LDU*

Recordemos que para una matriz, la factorización LU es el proceso de aplicación de la simplificación de Gauss, de modo que la matriz $L$ es una matriz triangular inferior con los coeficientes de transformación, y la matriz $U$ es la matriz superior con los elementos tras las transformaciones lineales.

Además, se puede hacer que $D$ sea una matriz diagonal con los valores de la diagonal de $U$, de modo que $LU$ = $LDD^{-1}U$, y si hacemos $U^* = D^{-1}U$ entonces tenemos $LDU^*$, donde $U^*$ sigue siendo una matriz diagonal superior, pero con la diagonal igual a 1.

A la hora de aplicar la factorización LU y LDU* se puede hacer una permutación de filas, de modo que en cada iteración se coge la fila con mayor valor (de entre las que no se han *procesado*) y se permuta, garantizando una solución siempre. También, es importante tener en mente que la factorización falla si algún elemento de la diagonal (desde el principio o durante la factorización) es 0, de modo que para solucionar ese caso se aplica la permutación.

Todas las permutaciones quedan recogidas en una matriz $P$, de modo que $$LU = LDU^* = PA$$

In [67]:
# Funciones asociadas a la factorización LU y LUD

def permutacion_matriz(U, fila_i, idx_max, verbose, P=None, r=None):
      if verbose:
        print(f'Permutamos fila {fila_i} con {idx_max}')
        print(f'U antes:\n {np.array(U)}')
        print(f'P antes:\n {np.array(P)}')

      if fila_i != idx_max:
        fila_origen, fila_destino = U[fila_i, :].copy(), U[idx_max, :].copy()
        U[idx_max, :], U[fila_i, :] = fila_origen, fila_destino
        if P is not None:
          fila_origen, fila_destino = P[fila_i, :].copy(), P[idx_max, :].copy()
          P[idx_max, :], P[fila_i, :] = fila_origen, fila_destino
        if r is not None:
          fila_origen, fila_destino = r[fila_i, :].copy(), r[idx_max, :].copy()
          r[idx_max, :], r[fila_i, :] = fila_origen, fila_destino

      if verbose:
         print(f'U despues:\n {np.array(U)}')
         print(f'P despues:\n {np.array(P)}')
      return U, P, r

def permutacion_L(L, perm, verbose):
  """
  Esta función la creo porque a la hora de hacer la permutación, hay que 
  permutar los elementos de L pero no directamente. Solo hay que seleccionar
  los elementos de la diagonal inferior que correspondan con el menor número de
  columnas. Por ejemplo, si permutamos las filas 3 y 5, se tienen que mover solo
  los elementos de las 2 primeras columnas (2 -> 3) para luego continuar 
  con las transformaciones del resto de columnas.
  """
  if verbose:
    print(f'L antes:\n {np.array(L)}')

  fila_origen, fila_destino = L[perm[0], :min(perm)].copy(), L[perm[1], :min(perm)].copy()
  L[perm[1], :min(perm)], L[perm[0], :min(perm)] = fila_origen, fila_destino

  if verbose:
    print(f'L despues:\n {np.array(L)}')
  return L 


def descomposicion_LU(m, rhs=None, verbose=True, permutar_max=False):
  '''
  Esta función realiza el algoritmo de triangulación de Gauss. Para ello vamos a
  ir aplicando paso a paso el algoritmo tal cual se hace manualmente, 
  y aplicamos los cambios de columnas necesarios si hay que aplicar permutaciones.
  Por defecto, si encontramos un 0 en la diagonal aplicamos permutar_max para esa
  fila, y devolvemos la matriz de permutaciones.
  '''

  if rhs is None:
    rhs = zeros(m.shape[0], 1)

  if verbose:
    print("La matriz M|X es  (X = 0) si no se ha introducido")
    print(np.concatenate((np.array(m), np.array(rhs)), axis=1))
  
  P, L, U, r = eye(m.shape[0]), eye(m.shape[0]), m.copy(), rhs.copy()
  lista_perms = []
  fila_i, err, err_max = 0, 0, 3
  

  while fila_i < m.shape[0] and err < err_max:
    if verbose:
      print(f'\n=====================================\nFila {fila_i}')
      print(f'A {fila_i}, {err}')

    if U[max(0, fila_i-1), max(0, fila_i-1)] == 0 or permutar_max:
      """
      Esta parte nos asegura que si un elemento diagonal es cero, permutamos
      la fila con su inmediata inferior y rehacemos los cálculos, 
      y así tener una configuración viable.
      En cualquier permutación tenemos que cambiar la L acorde con el cambio.
      Al hacer la permutación, con los nuevos valores, repasamos la matriz para 
      asegurarnos de que todos los puntos están cumplidos.

      Para la opción de permutar, buscamos un elemento de las filas no alteradas
      que sea el mayor. Si hay más de una fila, y una de ellas es la 
      actual, no aplica la permutación.
      """
      sub_mat = U[fila_i:, fila_i:]
      max_el = np.max(np.array(sub_mat))
      idx_max = np.min([i[0] for i in np.argwhere(np.array(sub_mat) == max_el)]) + fila_i # Cogemos el primer elemento si hay varios
      
      if U[max(0, fila_i-1), max(0, fila_i-1)] == 0:
        U, P, r = permutacion_matriz(U, max(0, fila_i-1), idx_max, verbose, P, r)
        L = permutacion_L(L, [max(0, fila_i-1), idx_max], verbose)
        fila_i = min(max(0, fila_i-1), idx_max)
        
        err += 1
        if verbose:
          print(f'Err {err}')
        continue

      elif permutar_max:
        if idx_max <= fila_i: # No es necesario hacer la permutación
          if verbose:
            print('El índice de permutación es igual a la fila a permutar.')
        else:
          U, P, r = permutacion_matriz(U, fila_i, idx_max, verbose, P, r)
          L = permutacion_L(L, [fila_i, idx_max], verbose)
          continue

    # Ahora aplicamos el algoritmo de calculo de filas:
    for columna_j in range(fila_i):
      a_ij = U[fila_i, columna_j] /  U[columna_j, columna_j]
      
      if a_ij != 0:
        L[fila_i, columna_j] = a_ij
        U[fila_i, :] = U[fila_i, :] - a_ij * U[columna_j, :]
        r[fila_i, :] = r[fila_i, :] - a_ij * r[columna_j, :]

      err = 0

      if verbose:
        print(f'||||||||||||||||||||||||\n Columna {columna_j}')
        print(f'a_{fila_i},{columna_j} = {a_ij}')
        print(f'P = \n{np.array(P)}')
        print(f'L = \n{np.array(L)}')
        print(f'U = \n{np.array(U)}')
        print(f'r = \n{np.array(r)}')

    fila_i += 1
  
  if err == err_max:
    print('Algo ha ido mal... mira el log.')

  if verbose:
        print(f'\/\/\/\/\/\/\/\/\/ FORMA FINAL')
        print(f'P = \n{np.array(P)}\n')
        print(f'L = \n{np.array(L)}\n')
        print(f'U = \n{np.array(U)}\n')
        print(f'r = \n{np.array(r)}\n')

  assert L * U == P * m

  return {'P': P, 'L': L, 'U': U, 'r': r}


def descomposicion_LDU(m, permutar_max=True, verbose=False):
  dict_LU = descomposicion_LU(m, permutar_max=permutar_max, verbose=verbose)
  L, U = dict_LU['L'], dict_LU['U']

  D = zeros(m.shape[0], m.shape[0])

  for i in range(U.shape[0]):
    D[i, i] = U[i, i]
  
  U = D.inv() * U

  if verbose:
        print(f'\/\/\/\/\/\/\/\/\/ RESULTS LDU*')
        print(f'L = \n{np.array(L)}\n')
        print(f'D = \n{np.array(D)}\n')
        print(f'U* = \n{np.array(U)}\n')

  assert L * D * U == dict_LU['P'] * m

  return {'P': dict_LU['P'], 'L': L, 'U': U, 'D': D}
  
  


In [31]:
M = mat(((1, 4, 4), (3, 2, 1), (2, 4, 1)))

In [ ]:
descomposicion_LU(M, permutar_max=False)

In [ ]:
descomposicion_LDU(M, permutar_max=False)

In [68]:
descomposicion_LU(M, rhs=ones(M.shape[0], 1), permutar_max=True)

La matriz M|X es  (X = 0) si no se ha introducido
[[1 4 4 1]
 [3 2 1 1]
 [2 4 1 1]]

Fila 0
A 0, 0
El índice de permutación es igual a la fila a permutar.

Fila 1
A 1, 0
Permutamos fila 1 con 2
U antes:
 [[1 4 4]
 [3 2 1]
 [2 4 1]]
P antes:
 [[1 0 0]
 [0 1 0]
 [0 0 1]]
U despues:
 [[1 4 4]
 [2 4 1]
 [3 2 1]]
P despues:
 [[1 0 0]
 [0 0 1]
 [0 1 0]]
L antes:
 [[1 0 0]
 [0 1 0]
 [0 0 1]]
L despues:
 [[1 0 0]
 [0 1 0]
 [0 0 1]]

Fila 1
A 1, 0
El índice de permutación es igual a la fila a permutar.
||||||||||||||||||||||||
 Columna 0
a_1,0 = 2
P = 
[[1 0 0]
 [0 0 1]
 [0 1 0]]
L = 
[[1 0 0]
 [2 1 0]
 [0 0 1]]
U = 
[[1 4 4]
 [0 -4 -7]
 [3 2 1]]
r = 
[[1]
 [-1]
 [1]]

Fila 2
A 2, 0
El índice de permutación es igual a la fila a permutar.
||||||||||||||||||||||||
 Columna 0
a_2,0 = 3
P = 
[[1 0 0]
 [0 0 1]
 [0 1 0]]
L = 
[[1 0 0]
 [2 1 0]
 [3 0 1]]
U = 
[[1 4 4]
 [0 -4 -7]
 [0 -10 -11]]
r = 
[[1]
 [-1]
 [-2]]
||||||||||||||||||||||||
 Columna 1
a_2,1 = 5/2
P = 
[[1 0 0]
 [0 0 1]
 [0 1 0]]
L = 
[

{'L': Matrix([
 [1,   0, 0],
 [2,   1, 0],
 [3, 5/2, 1]]), 'P': Matrix([
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0]]), 'U': Matrix([
 [1,  4,    4],
 [0, -4,   -7],
 [0,  0, 13/2]]), 'r': Matrix([
 [  1],
 [ -1],
 [1/2]])}

In [69]:
descomposicion_LDU(M, permutar_max=True)

{'D': Matrix([
 [1,  0,    0],
 [0, -4,    0],
 [0,  0, 13/2]]), 'L': Matrix([
 [1,   0, 0],
 [2,   1, 0],
 [3, 5/2, 1]]), 'P': Matrix([
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0]]), 'U': Matrix([
 [1, 4,   4],
 [0, 1, 7/4],
 [0, 0,   1]])}

### Factorización de Cholesky

La factorización de Cholesky es una factorización que genera una matriz triangular inferior $L$ tal que $A = LL^T$. Para que una matriz sea factorizable, tiene que cumplir que sus menores principales sean positivos, y que sea simétrica.

In [89]:
def cholesky(m, verbose=False):
  """
  Primero comprobamos que los menores sean positivos. Eso es equivalente que
  sus autovalores sean positivos. Por simplificar, si el determinante es negativo
  ya descartamos que sea factorizable, y saltamos el warning.
  """

  if m != m.T:
    print('AVISO! La matriz no es simétrica, y por tanto no factorizable por Cholesky.')

  if verbose:
    print(f"|M| es {m.det()}. Si es < 0, no es factorizable.")

  m_chol = zeros(m.shape[0], m.shape[1])

  for col_j in range(m.shape[0]):
    for row_i in range(col_j, m.shape[0]):
      if col_j == 0:
        if row_i == 0:
          m_chol[row_i, col_j] = sqrt(m[row_i, col_j])
        else:
          m_chol[row_i, col_j] = m[row_i, col_j] / m_chol[0, 0]
      
      else:
        if col_j == row_i:
          m_chol[row_i, col_j] = sqrt(m[col_j, col_j] - sum([m_chol[col_j, k] 
                                                      ** 2 for k in range(col_j)]))
        else:
          m_chol[row_i, col_j] = (m[row_i, col_j] - sum([m_chol[col_j, k] * m_chol[row_i, k] for k in range(col_j)]))/(m_chol[col_j, col_j])
      
  assert m_chol * m_chol.T == m, f"La matriz no es Cholesky-zable. \n L = \n {np.array(m_chol)} \n\n L*L.T = \n {np.array(m_chol * m_chol.T)}" 

  return m_chol

In [116]:
cholesky(MM)

AVISO! La matriz no es simétrica, y por tanto no factorizable por Cholesky.


AssertionError: ignored

In [92]:
M

Matrix([
[2, 7, 6],
[7, 4, 5],
[6, 5, 2]])

In [94]:
cholesky(M)

AVISO! La matriz no es simétrica, y por tanto no factorizable por Cholesky.


AssertionError: ignored

In [95]:
cholesky(M + M.T)

Matrix([
[    sqrt(2),                0,                0],
[7*sqrt(2)/2,     sqrt(82)*I/2,                0],
[  3*sqrt(2), 16*sqrt(82)*I/41, 12*sqrt(41)*I/41]])

In [97]:
M + M.T

Matrix([
[2, 7, 6],
[7, 4, 5],
[6, 5, 2]])

In [99]:
cholesky(M + M.T) * cholesky(M + M.T).T

Matrix([
[2, 7, 6],
[7, 4, 5],
[6, 5, 2]])

In [118]:
# Podemos hacer también Cholesky a una matriz con símbolos!

Mx = mat(((1, x, 1), (x, 2, 1), (1, 1, 3)))
Mx

Matrix([
[1, x, 1],
[x, 2, 1],
[1, 1, 3]])

In [119]:
cholesky(Mx)

Matrix([
[1,                      0,                                0],
[x,         sqrt(2 - x**2),                                0],
[1, (1 - x)/sqrt(2 - x**2), sqrt(-(1 - x)**2/(2 - x**2) + 2)]])

### Ortogonalización de Gram-Schmidt




In [185]:
def suma_columnas(lista):
  if len(lista) > 0:
    m = zeros(lista[0].shape[0], 1)
    for i in lista:
      m += i
    return m
  else:
    return 0

def gram_schmidt(m, verbose=False):
  """
  La ortogonalización produce una matriz ortogonalizada por columnas.
  p es la matriz ortogonal y p_norm es la ortonormal
  c es la matriz triangular tal que cij = aj·pi/||pi||^2 (los coeficientes de ortogonalización). Estos coeficientes se usan para la factorización QR.
  """

  p = zeros(m.shape[0], m.shape[1])

  p[:, 0] = m[:, 0]

  for col in range(1, m.shape[1]):
    p[:, col] = m[:, col] - suma_columnas([(m[:, col].T * p[:, i])[0]/(p[:, i].T * p[:, i])[0] * p[:, i] for i in range(0, col)])

  if verbose:
      print(f"La matriz ortogonal es \n {m_gs}")


  p_norm = zeros(m.shape[0], m.shape[1])
  for col in range(p.shape[1]):
    p_norm[:, col] = p[:, col] / (p[:, col].T * p[:, col])[0]
  
  if verbose:
    print(f"La matriz ortonormal es \n {p_norm}")


  c = zeros(m.shape[0], m.shape[1])
  for col in range(1, m.shape[1]):
    for row in range(0, col):
      c[row, col] = (m[:, col].T * p[:, row])[0]/(p[:, row].T * p[:, row])[0]

  return {'P': p, 'Pn': p_norm, 'c': c} 


In [186]:
GS = gram_schmidt(M)
GS['P']

Matrix([
[1,  19/7, 52/45],
[3, -13/7, 26/45],
[2,  10/7, -13/9]])

In [187]:
GS['Pn']

Matrix([
[1/14,  19/90,  4/13],
[3/14, -13/90,  2/13],
[ 1/7,    1/9, -5/13]])

In [188]:
GS['c']

Matrix([
[0, 9/7,  9/14],
[0,   0, 73/90],
[0,   0,     0]])

In [159]:
GS[:, 1].T * GS[:, 2]

Matrix([[0]])

In [162]:
Mx = mat(((1, 2, 3), (1, x, 1), (0, 0, 3)))

In [163]:
gram_schmidt(Mx)

Matrix([
[1/2, (1 - x/2)/((1 - x/2)**2 + (x/2 - 1)**2), (-(1 - x/2)*(2 - x)/((1 - x/2)**2 + (x/2 - 1)**2) + 1)/((-(1 - x/2)*(2 - x)/((1 - x/2)**2 + (x/2 - 1)**2) + 1)**2 + (-(2 - x)*(x/2 - 1)/((1 - x/2)**2 + (x/2 - 1)**2) - 1)**2 + 9)],
[1/2, (x/2 - 1)/((1 - x/2)**2 + (x/2 - 1)**2), (-(2 - x)*(x/2 - 1)/((1 - x/2)**2 + (x/2 - 1)**2) - 1)/((-(1 - x/2)*(2 - x)/((1 - x/2)**2 + (x/2 - 1)**2) + 1)**2 + (-(2 - x)*(x/2 - 1)/((1 - x/2)**2 + (x/2 - 1)**2) - 1)**2 + 9)],
[  0,                                       0,                                                      3/((-(1 - x/2)*(2 - x)/((1 - x/2)**2 + (x/2 - 1)**2) + 1)**2 + (-(2 - x)*(x/2 - 1)/((1 - x/2)**2 + (x/2 - 1)**2) - 1)**2 + 9)]])

### Factorización QR

La factorización QR consiste en transformar $A = QR$ donde $Q$ es ortogonal y $R$ es triangular superior. 

Si $P$ es la matriz ortogonalizada, $C$ es la matriz con los factores de ortonormalización (para Gram-Schmidt, por ejemplo, es $m_{ij} = \frac{a^j\cdot p^i}{\vert\vert p^{i}\vert\vert^2}$ y $D$ es la matriz de las normas de los vectores ortogonales ($\vert\vert p^i\vert\vert$), entonces se tiene que:
$$Q = PD^{-1}$$
$$R = D(I + C)$$

In [209]:
def factorizacion_QR(m, verbose=True):
  dict_gs = gram_schmidt(m)
  P, C = dict_gs['P'], dict_gs['c']
  
  D = zeros(m.shape[0], m.shape[1])
  for col in range(m.shape[0]):
    D[col, col] = sqrt((P[:, col].T * P[:, col])[0])
  
  Q = P * (D ** (-1))
  R = D * (eye(m.shape[0]) + C)
  
  if verbose:
    print(f'Q es \n{np.array(Q)}\nR es \n{np.array(R)}\nD es \n{np.array(D)}')

  assert m == Q * R

  return {'Q': Q, 'R': R, 'D': D} 

In [202]:
M = mat(((2, -1, 0), (0, 0, -2), (0, 2, -1)))

In [210]:
factorizacion_QR(M)

Q es 
[[1 0 0]
 [0 0 -1]
 [0 1 0]]
R es 
[[2 -1 0]
 [0 2 -1]
 [0 0 2]]
D es 
[[2 0 0]
 [0 2 0]
 [0 0 2]]


{'D': Matrix([
 [2, 0, 0],
 [0, 2, 0],
 [0, 0, 2]]), 'Q': Matrix([
 [1, 0,  0],
 [0, 0, -1],
 [0, 1,  0]]), 'R': Matrix([
 [2, -1,  0],
 [0,  2, -1],
 [0,  0,  2]])}